In [1]:
from omegaconf import OmegaConf

In [2]:
config = OmegaConf.load('/home/aiteam/tykim/generative_model/framework/diffusion/ControlNet/models/cldm_v15.yaml')

In [3]:
config

{'model': {'target': 'cldm.cldm.ControlLDM', 'params': {'linear_start': 0.00085, 'linear_end': 0.012, 'num_timesteps_cond': 1, 'log_every_t': 200, 'timesteps': 1000, 'first_stage_key': 'jpg', 'cond_stage_key': 'txt', 'control_key': 'hint', 'image_size': 64, 'channels': 4, 'cond_stage_trainable': False, 'conditioning_key': 'crossattn', 'monitor': 'val/loss_simple_ema', 'scale_factor': 0.18215, 'use_ema': False, 'only_mid_control': False, 'control_stage_config': {'target': 'cldm.cldm.ControlNet', 'params': {'image_size': 32, 'in_channels': 4, 'hint_channels': 3, 'model_channels': 320, 'attention_resolutions': [4, 2, 1], 'num_res_blocks': 2, 'channel_mult': [1, 2, 4, 4], 'num_heads': 8, 'use_spatial_transformer': True, 'transformer_depth': 1, 'context_dim': 768, 'use_checkpoint': True, 'legacy': False}}, 'unet_config': {'target': 'cldm.cldm.ControlledUnetModel', 'params': {'image_size': 32, 'in_channels': 4, 'out_channels': 4, 'model_channels': 320, 'attention_resolutions': [4, 2, 1], 'nu

In [8]:
def instantiate_from_config(config):
    if not "target" in config:
        if config == '__is_first_stage__':
            return None
        elif config == "__is_unconditional__":
            return None
        raise KeyError("Expected key `target` to instantiate.")
    print(config['target'])


In [9]:
instantiate_from_config(config.model)

cldm.cldm.ControlLDM


In [12]:
config.model.get("params", dict())

{'linear_start': 0.00085, 'linear_end': 0.012, 'num_timesteps_cond': 1, 'log_every_t': 200, 'timesteps': 1000, 'first_stage_key': 'jpg', 'cond_stage_key': 'txt', 'control_key': 'hint', 'image_size': 64, 'channels': 4, 'cond_stage_trainable': False, 'conditioning_key': 'crossattn', 'monitor': 'val/loss_simple_ema', 'scale_factor': 0.18215, 'use_ema': False, 'only_mid_control': False, 'control_stage_config': {'target': 'cldm.cldm.ControlNet', 'params': {'image_size': 32, 'in_channels': 4, 'hint_channels': 3, 'model_channels': 320, 'attention_resolutions': [4, 2, 1], 'num_res_blocks': 2, 'channel_mult': [1, 2, 4, 4], 'num_heads': 8, 'use_spatial_transformer': True, 'transformer_depth': 1, 'context_dim': 768, 'use_checkpoint': True, 'legacy': False}}, 'unet_config': {'target': 'cldm.cldm.ControlledUnetModel', 'params': {'image_size': 32, 'in_channels': 4, 'out_channels': 4, 'model_channels': 320, 'attention_resolutions': [4, 2, 1], 'num_res_blocks': 2, 'channel_mult': [1, 2, 4, 4], 'num_he

In [19]:
def p(linear_start, **kwargs):
    print(linear_start)
    print(type(kwargs))
    print(kwargs)

In [20]:
p(**config.model.get("params", dict()))

0.00085
<class 'dict'>
{'linear_end': 0.012, 'num_timesteps_cond': 1, 'log_every_t': 200, 'timesteps': 1000, 'first_stage_key': 'jpg', 'cond_stage_key': 'txt', 'control_key': 'hint', 'image_size': 64, 'channels': 4, 'cond_stage_trainable': False, 'conditioning_key': 'crossattn', 'monitor': 'val/loss_simple_ema', 'scale_factor': 0.18215, 'use_ema': False, 'only_mid_control': False, 'control_stage_config': {'target': 'cldm.cldm.ControlNet', 'params': {'image_size': 32, 'in_channels': 4, 'hint_channels': 3, 'model_channels': 320, 'attention_resolutions': [4, 2, 1], 'num_res_blocks': 2, 'channel_mult': [1, 2, 4, 4], 'num_heads': 8, 'use_spatial_transformer': True, 'transformer_depth': 1, 'context_dim': 768, 'use_checkpoint': True, 'legacy': False}}, 'unet_config': {'target': 'cldm.cldm.ControlledUnetModel', 'params': {'image_size': 32, 'in_channels': 4, 'out_channels': 4, 'model_channels': 320, 'attention_resolutions': [4, 2, 1], 'num_res_blocks': 2, 'channel_mult': [1, 2, 4, 4], 'num_head

In [32]:
import torch
import lovely_tensors as lt
import math
import numpy as np


lt.monkey_patch()
timesteps= 100
s = 0.008

In [26]:
steps = timesteps + 1
x = torch.linspace(0, timesteps, steps)
alphas_cumprod = torch.cos(((x / timesteps) + s) / (1 + s) * math.pi * 0.5) ** 2
alphas_cumprod

tensor[101] x∈[1.911e-15, 1.000] μ=0.496 σ=0.356

In [28]:
alphas_cumprod = alphas_cumprod / alphas_cumprod[0]
alphas_cumprod

tensor[101] x∈[1.911e-15, 1.000] μ=0.496 σ=0.356

In [29]:
betas = 1 - (alphas_cumprod[1:] / alphas_cumprod[:-1])
betas

tensor[100] x∈[0.001, 1.000] μ=0.079 σ=0.147

In [33]:
np.clip(betas, 0.0001, 0.9999)

tensor[100] x∈[0.001, 1.000] μ=0.079 σ=0.147

In [34]:
torch.clamp

AttributeError: module 'torch' has no attribute 'clip'

In [35]:
!pip list

Package                  Version
------------------------ ------------
absl-py                  1.4.0
accelerate               0.14.0
addict                   2.4.0
aiofiles                 22.1.0
aiohttp                  3.8.3
aiohttp-cors             0.7.0
aiorwlock                1.3.0
aiosignal                1.3.1
aiosqlite                0.18.0
antlr4-python3-runtime   4.8
anyio                    3.6.2
appdirs                  1.4.4
argon2-cffi              21.3.0
argon2-cffi-bindings     21.2.0
asttokens                2.2.1
async-timeout            4.0.2
attrs                    22.2.0
audiolm-pytorch          0.17.0
audioread                3.0.0
Babel                    2.12.1
backcall                 0.2.0
Backtesting              0.3.3
bayesian-optimization    1.3.1
bcrypt                   4.0.1
beartype                 0.12.0
beautifulsoup4           4.12.0
bitarray                 2.7.3
bleach                   6.0.0
blessed                  1.20.0
blobfile             